In [116]:
import pandas as pd
import numpy as np

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк. Обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [118]:
tr_types_tbl = pd.read_csv('/content/drive/MyDrive/data/tr_types.csv', delimiter=';')
gender_train_tbl = pd.read_csv('/content/drive/MyDrive/data/gender_train.csv')
tr_mcc_codes_tbl = pd.read_csv('/content/drive/MyDrive/data/tr_mcc_codes.csv', delimiter=';')
transactions_tbl = pd.read_csv('/content/drive/MyDrive/data/transactions.csv', nrows=1000000)

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

In [119]:
tr_type_smpl = transactions_tbl['tr_type'].sample(1000)

In [120]:

mapping = dict(tr_types_tbl[['tr_type', 'tr_description']].values)
mapped = tr_type_smpl.map(mapping)

filtered_mapping = mapped.str.contains('POS|АТМ', regex=True, na=False)

print(len(mapped[filtered_mapping]) / len(tr_type_smpl))

0.61


## Задание 2 


1. Для столюца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [121]:
tr_type_freq = transactions_tbl['tr_type'].value_counts()
tr_type_top_ten = tr_type_freq[:10]
print(tr_type_top_ten)

1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
2370     49830
7010     28972
7030     26078
7071     15781
1100     15476
Name: tr_type, dtype: int64


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [205]:
cust_sum_inc = transactions_tbl[transactions_tbl['amount'] > 0].groupby(['customer_id'])['amount'].sum()
cust_sum_exp = transactions_tbl[transactions_tbl['amount'] < 0].groupby(['customer_id'])['amount'].sum()

max_sum_id = cust_sum_inc.idxmax()
min_sum_id = cust_sum_exp.idxmin()

print("1. ", max_sum_id)
print("2. ", min_sum_id)
print("3. ", round(abs(cust_sum_inc.get(max_sum_id) - abs(cust_sum_exp.get(min_sum_id))), 3))

1.  70780820
2.  70780820
3.  1837317.98


## Задание 4
1. Найдите среднее арифметическое и медиану по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по всем типам транзакций для клиентов из задания 3

In [123]:
tr_type_top_ten_ind = tr_type_top_ten.index
tr_type_top = transactions_tbl[transactions_tbl['tr_type'].isin(tr_type_top_ten_ind)].groupby(['tr_type'])['amount']

tr_cust = transactions_tbl[transactions_tbl['customer_id'].isin((max_sum_id, min_sum_id))].groupby(['tr_type'])['amount']

print('1. ', 'mean: ', tr_type_top.mean(), '\nmedian: ', tr_type_top.median() )

print('2. ', 'mean: ', tr_cust.mean(), '\nmedian: ', tr_cust.median())

1.  mean:  tr_type
1010    -19784.748640
1030     -5320.980222
1100    -44061.827262
1110    -32119.330371
2010   -136077.629325
2370   -205418.249032
7010    276391.789596
7030     86104.332909
7070     65569.831700
7071     66806.826623
Name: amount, dtype: float64 
median:  tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64
2.  mean:  tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
7070    1.384821e+04
7071    1.860057e+04
7074    2.281159e+04
Name: amount, dtype: float64 
median:  tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      12992.620
7034  

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [124]:
transactions_tbl = pd.merge(transactions_tbl, gender_train_tbl, how='left')
transactions_tbl = pd.merge(transactions_tbl, tr_mcc_codes_tbl, how='inner')
transactions_tbl = pd.merge(transactions_tbl, tr_types_tbl, how='inner')
transactions_tbl.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [204]:
mean_inc = transactions_tbl[transactions_tbl['amount'] > 0].groupby(['gender'])['amount'].mean()
mean_exp = transactions_tbl[transactions_tbl['amount'] < 0].groupby(['gender'])['amount'].mean()

# print(mean_inc)
print("Mean expense difference: \n", round(abs(mean_exp.get(0) - mean_exp.get(1)), 3))
print("Mean income difference: \n", round(abs(mean_inc.get(0) - mean_inc.get(1)), 3))

Mean expense difference: 
 32718.055
Mean income difference: 
 63366.571


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income". 
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.

In [126]:
sum_exp = transactions_tbl[transactions_tbl['amount'] < 0].groupby(['gender', 'mcc_code'])['amount'].sum()


print(abs(sum_exp.get(1) - sum_exp.get(0)).nlargest())

mcc_code
6011    3.085862e+09
4829    2.499153e+09
6010    2.208580e+08
5541    1.554326e+08
5511    7.352877e+07
Name: amount, dtype: float64


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте ночные поступления для мужчин и женщин. Ночные - это примерно 00-06 часов. 

In [127]:
transactions_tbl['tr_hour'] = transactions_tbl['tr_datetime'].map(lambda x: int(x.split()[1][:2]))

night_income = transactions_tbl[(transactions_tbl['amount'] >= 0) &\
                                (transactions_tbl['tr_hour'] >= 0) &\
                                (transactions_tbl['tr_hour'] <= 6)].groupby(['gender'])['amount'].sum() 
print(night_income)

gender
0.0    5.806437e+08
1.0    9.226917e+08
Name: amount, dtype: float64


In [202]:
# additional task
transactions_tbl['tr_hour'] = transactions_tbl['tr_datetime'].map(lambda x: int(x.split()[1][:2]))
night_tr = transactions_tbl[(transactions_tbl['tr_hour'] >= 0) &\
                                (transactions_tbl['tr_hour'] <= 6)]
night_cust = night_tr['customer_id'].value_counts()

day_tr = transactions_tbl[(transactions_tbl['tr_hour'] >= 11) &\
                                (transactions_tbl['tr_hour'] <= 17)]
day_cust = day_tr['customer_id'].value_counts()


night_larger_day = ((night_cust - day_cust) > 0).dropna()
customers_id = day_cust[night_larger_day].index

gender_solution = gender_train_tbl[gender_train_tbl['customer_id'].isin(customers_id)]

print(gender_solution['gender'].value_counts())



1    83
0    72
Name: gender, dtype: int64
